In [17]:
# !pip install "huggingface_hub[hf_transfer]" gguf

# !git clone https://github.com/ggerganov/llama.cpp.git
# !python3 -m pip install -r llama.cpp/requirements.txt
# !make -C llama.cpp -j4

import os
from huggingface_hub import snapshot_download

## Uncomment the following line and replace YOUR_API_KEY with your Hugging Face API key
# os.environ["HUGGINGFACE_API_KEY"] = "YOUR_API_KEY"

os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

if os.path.exists("./Meta-Llama-3.1-8B-Instruct"):
    print("Directory already exists")
else:
    snapshot_download(
        repo_id="meta-llama/Meta-Llama-3.1-8B-Instruct", 
        token=os.environ["HUGGINGFACE_API_KEY"],
        local_dir="./Meta-Llama-3.1-8B-Instruct",
    )

---

In [19]:
import os
import subprocess

if os.path.exists("./llama.cpp/llama-quantize"):
    print("llama-quantize already exists")
else:
    # Run the Python script to convert the model
    subprocess.run([
        "python3", "llama.cpp/convert_hf_to_gguf.py", "Meta-Llama-3.1-8B-Instruct"
    ], check=True)

    # Run the llama-quantize command
    subprocess.run([
        "./llama.cpp/llama-quantize",
        "Meta-Llama-3.1-8B-Instruct/Meta-Llama-3.1-8B-Instruct-F16.gguf",
        "Meta-Llama-3.1-8B-Instruct/Meta-Llama-3.1-8B-Instruct--q4_0.bin",
        "q4_0"
    ], check=True)

llama-quantize already exists


Serving the quantized model using `llama-server`

In [20]:
!./llama.cpp/llama-server -m Meta-Llama-3.1-8B-Instruct/Meta-Llama-3.1-8B-Instruct-F16.gguf --port 1905

# Basic web UI can be accessed via browser: http://localhost:1905
# Chat completion endpoint: http://localhost:1905/v1/chat/completions

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


build: 3918 (4c42f93b) with cc (Debian 10.2.1-6) 10.2.1 20210110 for x86_64-linux-gnu
system info: n_threads = 16, n_threads_batch = 16, total_threads = 32

system_info: n_threads = 16 (n_threads_batch = 16) / 32 | AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 1 | NEON = 0 | SVE = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | RISCV_VECT = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 1 | 

main: HTTP server is listening, hostname: 127.0.0.1, port: 1905, http threads: 31
main: loading model
llama_model_loader: loaded meta data with 33 key-value pairs and 292 tensors from Meta-Llama-3.1-8B-Instruct/Meta-Llama-3.1-8B-Instruct-F16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:

In [1]:
!./llama.cpp/llama-cli \
  -m Meta-Llama-3.1-8B-Instruct/Meta-Llama-3.1-8B-Instruct--q4_0.bin \
  -n 256 --repeat_penalty 1.0 --color -i -r "User:" \
  -f llama.cpp/prompts/chat-with-bob.txt

build: 3918 (4c42f93b) with cc (Debian 10.2.1-6) 10.2.1 20210110 for x86_64-linux-gnu
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_loader: loaded meta data with 33 key-value pairs and 292 tensors from Meta-Llama-3.1-8B-Instruct/Meta-Llama-3.1-8B-Instruct--q4_0.bin (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Meta Llama 3.1 8B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Meta-Llama-3.1
llama_model_loader: - kv   5:              

In [ ]:
# !CMAKE_ARGS="-DGGML_CUDA=on" pip install llama-cpp-python --upgrade

In [3]:
import torch
torch.cuda.is_available()

True

In [2]:
import os
os.environ["LLAMA_CPP_LIB"] = "/space/hotel/phit/projects/arXivRAG/notebooks/llama.cpp/libllama.so"
os.environ["LLAMA_CUBLAS"] = "on"

In [3]:
from llama_cpp import Llama

llm = Llama(
      model_path="./Meta-Llama-3.1-8B-Instruct/Meta-Llama-3.1-8B-Instruct-F16.gguf",
      n_gpu_layers=-1, # Uncomment to use GPU acceleration
      # seed=1337, # Uncomment to set a specific seed
      # n_ctx=2048, # Uncomment to increase the context window
)

llama_model_loader: loaded meta data with 33 key-value pairs and 292 tensors from ./Meta-Llama-3.1-8B-Instruct/Meta-Llama-3.1-8B-Instruct-F16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Meta Llama 3.1 8B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Meta-Llama-3.1
llama_model_loader: - kv   5:                         general.size_label str              = 8B
llama_model_loader: - kv   6:                            general.license str              = llama3.1
llama_mod

llama_model_loader: - kv  26:                      tokenizer.ggml.tokens arr[str,128256]  = ["!", "\"", "#", "$", "%", "&", "'", ...
llama_model_loader: - kv  27:                  tokenizer.ggml.token_type arr[i32,128256]  = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
llama_model_loader: - kv  28:                      tokenizer.ggml.merges arr[str,280147]  = ["Ġ Ġ", "Ġ ĠĠĠ", "ĠĠ ĠĠ", "...
llama_model_loader: - kv  29:                tokenizer.ggml.bos_token_id u32              = 128000
llama_model_loader: - kv  30:                tokenizer.ggml.eos_token_id u32              = 128009
llama_model_loader: - kv  31:                    tokenizer.chat_template str              = {{- bos_token }}\n{%- if custom_tools ...
llama_model_loader: - kv  32:               general.quantization_version u32              = 2
llama_model_loader: - type  f32:   66 tensors
llama_model_loader: - type  f16:  226 tensors
llm_load_vocab: special tokens cache size = 256
llm_load_vocab: token to piece cache size = 0

In [4]:
output = llm(
      "Q: Name the planets in the solar system? A: ", # Prompt
      max_tokens=32, # Generate up to 32 tokens, set to None to generate up to the end of the context window
      stop=["Q:", "\n"], # Stop generating just before the model would generate a new question
      echo=True # Echo the prompt back in the output
) # Generate a completion, can also call create_completion
print(output)

llama_perf_context_print:        load time =    2123.39 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    14 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    31 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   22792.88 ms /    45 tokens


{'id': 'cmpl-b0929ce1-47ae-4db6-ac09-ba48b610069e', 'object': 'text_completion', 'created': 1729052507, 'model': './Meta-Llama-3.1-8B-Instruct/Meta-Llama-3.1-8B-Instruct-F16.gguf', 'choices': [{'text': 'Q: Name the planets in the solar system? A:  Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune, Pluto (no longer a planet) Mercury is the closest to the sun and', 'index': 0, 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 14, 'completion_tokens': 32, 'total_tokens': 46}}


## AWQ (Adaptive Weight Quantization)
Not work as it requires CUDA >= 11.8

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AwqConfig

model_id = "hugging-quants/Meta-Llama-3.1-8B-Instruct-AWQ-INT4"

quantization_config = AwqConfig(
    bits=4,
    fuse_max_seq_len=512, # Note: Update this as per your use-case
    do_fuse=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
  model_id,
  torch_dtype=torch.float16,
  low_cpu_mem_usage=True,
  device_map="auto",
  quantization_config=quantization_config
)

prompt = [
  {"role": "system", "content": "You are a helpful assistant, that responds as a pirate."},
  {"role": "user", "content": "What's Deep Learning?"},
]
inputs = tokenizer.apply_chat_template(
  prompt,
  tokenize=True,
  add_generation_prompt=True,
  return_tensors="pt",
  return_dict=True,
).to("cuda")

outputs = model.generate(**inputs, do_sample=True, max_new_tokens=256)
print(tokenizer.batch_decode(outputs[:, inputs['input_ids'].shape[1]:], skip_special_tokens=True)[0])

/space/hotel/phit/miniconda3/envs/llm/lib/python3.10/site-packages/transformers/quantizers/auto.py:182: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.However, loading attributes (e.g. ['version', 'fuse_max_seq_len', 'exllama_config', 'modules_to_fuse', 'do_fuse']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

NameError: name 'awq_ext' is not defined

## GPTQ (GPT Quantization)

In [ ]:
import torch
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

quantize_config = BaseQuantizeConfig(
    bits=4,  # quantize model to 4-bit
    group_size=128,  # it is recommended to set the value to 128
    desc_act=True,  # set to False can significantly speed up inference but the perplexity may slightly bad
    sym=True,  # using symmetric quantization so that the range is symmetric allowing the value 0 to be precisely represented (can provide speedups)
    damp_percent=0.1,  # see https://github.com/AutoGPTQ/AutoGPTQ/issues/196
)

model_id = "hugging-quants/Meta-Llama-3.1-8B-Instruct-GPTQ-INT4"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoGPTQForCausalLM.from_quantized(
  model_id,
  torch_dtype=torch.float16,
  device="cuda:0",
  # low_cpu_mem_usage=True,
  # device_map="auto", # or "auto"
  quantize_config=quantize_config
)

WARNING - Exllamav2 kernel is not installed, reset disable_exllamav2 to True. This may because you installed auto_gptq using a pre-build wheel on Windows, in which exllama_kernels are not compiled. To use exllama_kernels to further speedup inference, you can re-install auto_gptq from source.
WARNING - CUDA kernels for auto_gptq are not installed, this will result in very slow inference speed. This may because:
1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.
/space/hotel/phit/miniconda3/envs/llm/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
INFO - The layer lm_head is not quantized.
Setting `pad_token_id` to 

["system\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nYou are a helpful assistant, that responds as a pirate.user\n\nWhat's Deep Learning?assistant\n\nArrr, ye landlubbers be wantin' to know about Deep Learning, eh?\n\nDeep Learning be a subset o' Machine Learnin', which be a powerful tool fer processin' and makin' sense o' loads o' data. 'Tis like havin' a trusty parrot that sits on yer shoulder, gazin' over yer shoulder, and squawkin' out clever insights!\n\nDeep Learning be all about Artificial Neural Networks, the brain-like computers that mimic how swabbies (our brains) think and learn. Just as our brains be composed o' layers o' neurons, so too are the computers in Deep Learning. Each layer be a set o' nodes or 'perceptrons' that process and transform the data in different ways.\n\nNow, ye might be wonderin' what kind o' tricks these computer swabbies be servin' up. Well, matey, some o' the most impressive feats o' Deep Learning be:\n\n1. **Image Recogniti

In [ ]:
prompt = [
  {"role": "system", "content": "You are a helpful assistant, that responds as a pirate."},
  {"role": "user", "content": "What's Deep Learning?"},
]
inputs = tokenizer.apply_chat_template(
  prompt,
  tokenize=True,
  add_generation_prompt=True,
  return_tensors="pt",
  return_dict=True,
).to("cuda")

outputs = model.generate(**inputs, do_sample=True, max_new_tokens=256)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


["system\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nYou are a helpful assistant, that responds as a pirate.user\n\nWhat's Deep Learning?assistant\n\nArrr, ye landlubbers, gather 'round. Ye be wantin' to know about Deep Learnin', eh?  Well, settle yerselves down with a pint o' grog and listen close.\n\nDeep Learnin' be a type o' machine learnin' that's as deep as the ocean itself. It's a form o' artificial intelligence that involves learnin' complex patterns in data using multiple layers o' artificail neural networks.\n\nThink o' a ship's hull, matey. At the bottom o' the hull, ye have the raw data, like a vast ocean full o' fishin' nets and anchors. Then, ye got the hidden layers o' neurons, like the sturdy ribs o' the ship, that help to process the data and extract valuable insights from it.\n\nThese neural networks be trained using lots o' data, like a scurvy dog gatherin' treasure from the depths o' the sea. The more data ye feed 'em, the stronger they beco

## Final Version

In [4]:
!echo $BNB_CUDA_VERSION

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Old version

```shell
    # https://github.com/bitsandbytes-foundation/bitsandbytes/issues/109
    git clone https://github.com/TimDettmers/bitsandbytes.git
    cd bitsandbytes
    CUDA_VERSION=116 make cuda11x
    python setup.py install
```

New version
```shell
    # New version
    git clone https://github.com/bitsandbytes-foundation/bitsandbytes.git && cd bitsandbytes/
    pip install -r requirements-dev.txt
    cmake -DCOMPUTE_BACKEND=cuda -S .
    make
    pip install -e .   # `-e` for "editable" install, when developing BNB (otherwise leave that out)

    # Check if the installation is successful
    python -m bitsandbytes

    # https://github.com/bitsandbytes-foundation/bitsandbytes/issues/156
    cp /mnt/net/i2x256-ai03/hotel/phit/.local/bitsandbytes/build/lib/bitsandbytes/libbitsandbytes_cuda116.so /space/hotel/phit/miniconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so
    conda install cudatoolkit=11.6 -y
```

THis will work 

```cmd
    pip install --force-reinstall 'https://github.com/bitsandbytes-foundation/bitsandbytes/releases/download/0.39.1/bitsandbytes-0.39.1-py3-none-any.whl'
```

In [9]:
import torch
import torch.nn as nn

from bitsandbytes.nn import Linear4bit

fp16_model = nn.Sequential(
    nn.Linear(64, 64),
    nn.Linear(64, 64)
)

quantized_model = nn.Sequential(
    Linear4bit(64, 64),
    Linear4bit(64, 64)
)

quantized_model.load_state_dict(fp16_model.state_dict())
quantized_model = quantized_model.to(0) # Quantization happens here

In [6]:
import bitsandbytes
print(dir(bitsandbytes))  # This will show available modules/attributes

['__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'autograd', 'cextension', 'consts', 'cuda_specs', 'functional', 'nn', 'optim', 'triton', 'utils']


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig  

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.bfloat16,
                                         bnb_4bit_use_double_quant=True,
                                         bnb_4bit_quant_type= "nf4"
                                         )


tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
  model_id,
  torch_dtype=torch.float16,
  low_cpu_mem_usage=True,
  device_map="cuda",
  quantization_config=quantization_config
)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /space/hotel/phit/miniconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda116.so
CUDA SETUP: CUDA runtime path found: /space/hotel/phit/miniconda3/envs/llm/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 116
CUDA SETUP: Loading binary /space/hotel/phit/miniconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda116.so...


/space/hotel/phit/miniconda3/envs/llm/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/space/hotel/phit/miniconda3/envs/llm/lib/libcudart.so.11.0'), PosixPath('/space/hotel/phit/miniconda3/envs/llm/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


In [3]:
from transformers import TextStreamer
prompt = [
  {"role": "system", "content": "You are a helpful assistant, that responds as a pirate."},
  {"role": "user", "content": "What's Deep Learning?"},
]
streamer = TextStreamer(tokenizer)

inputs = tokenizer.apply_chat_template(
  prompt,
  tokenize=True,
  add_generation_prompt=False,
  return_tensors="pt",
  return_dict=True,
).to("cuda")

outputs = model.generate(**inputs, streamer=streamer, do_sample=True, max_new_tokens=256)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant, that responds as a pirate.<|eot_id|><|start_header_id|>user<|end_header_id|>

What's Deep 

Learning?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Yer lookin' fer a definition o' Deep Learnin', eh?  Well, matey, settle yerself down with a pint o' grog and listen close.

Deep Learnin' be a branch o' Machine Learnin' that uses artificial neural networks to analyze and interpret data. It's called "deep" because these networks have multiple layers, like the layers o' an onion (but not as tear-inducin' as that, savvy?). Each layer processes the data in a different way, makin' it possible to learn complex patterns and relationships.

Think o' it like a pirate's treasure map. Ye start with a big ol' mess o' data (the treasure), and then ye use multiple layers o' processing (the map) to figure out where the treasure be hidin'. The more layers ye have, the better ye can navigate the map and find the loot!

Now, I know what ye be thinkin', "Pirate, what about the different types o' Deep Learnin'?"  Well, matey, there be several, including:

1. Supervised Learnin': Ye have a 